Showing how the framework can be loaded:

In [1]:
# built-in imports
import time

# package imports
from fiatmodel import Calibration

Testing the interface

In [2]:
# defining MESH calibration parameter bounds
class_dict_bounds = {
    1: {
        'sno': [0.0, 0.5],
        'rhos': [0.0, 0.5],
        'clay1': [10, 50],
        'sand1': [30, 70],
        'clay3': [20, 30],
    },
    3: {
        'lamx': [0.0, 1.0],
        'cmas': [0.0, 1.0],
        'clay2': [20, 60],
        'clay1': [0, 10],
    },
}

hydrology_dict_bounds = {
    1: {
        'zsnl': [0, 10],
        'zplg': [5, 15]
    },
    12: {
        'zpls': [0, 42],
        'zsnl': [0, 10],
        'zplg': [5, 15]
    },
}

routing_dict_bounds = {
    1: {
        'r2n': [0.0, 1.0],
        'r1n': [0.0, 1.0]
    },
    3: {
        'pwr': [0.0, 1.0],
        'flz': [0.0, 1.0]
    },
}

In [3]:
c = Calibration(
    calibration_software = 'ostrich',
    model_software = 'mesh',
    calibration_config = {
        'random_seed': int(time.time()),
        'algorithm': 'DDS',
        'algorithm_specs': { # refer to software manual for the keys
            'PerturbationValue': 0.2,
            'MaxIteration': 10_000,
            'UseRandomParamValue': None,
        },
        'metric': 'kge_2012', # refer to HydrErr documentation for valid keys, this needs more work
    },
    model_config = {
        'instance_path': './wolf-creek-research-basin/mesh/',
        'parameter_bounds': {
            'class': class_dict_bounds,
            'hydrology': hydrology_dict_bounds,
            'routing': routing_dict_bounds,
        },
    },
    observation_config = {
    },
)

In [4]:
c.model.analyze()

In [5]:
c.model.prepare()

In [6]:
c.model.templated_parameters['class']

{1: {'class': 'needleleaf',
  'fcan': 1.0,
  'lamx': 1.27,
  'lnz0': 0.166,
  'lamn': 0.975,
  'alvc': 0.044,
  'cmas': 4.718,
  'alic': 0.169,
  'root': 1.21,
  'rsmn': 212.33,
  'qa50': 0.0,
  'vpda': 0.46,
  'vpdb': 0.0,
  'psga': 100.0,
  'psgb': 0.0,
  'drn': 1.0,
  'sdep': 2.0,
  'fare': 1.0,
  'dd': 50.0,
  'xslp': 0.03,
  'xdrainh': 0.37,
  'mann': 0.11,
  'ksat': 0.446,
  'mid': 'Needleleaf Forest (temperate)',
  'sand1': '_1SAND1',
  'sand2': 81.73,
  'sand3': 81.73,
  'clay1': '_1CLAY1',
  'clay2': 10.53,
  'clay3': '_1CLAY3',
  'orgm1': 0.0,
  'orgm2': 0.0,
  'orgm3': 0.0,
  'tbar1': 4.0,
  'tbar2': 2.0,
  'tbar3': 1.0,
  'tcan': 4.0,
  'tsno': 0.0,
  'tpnd': 4.0,
  'thlq1': 0.25,
  'thlq2': 0.15,
  'thlq3': 0.04,
  'thic1': 0.0,
  'thic2': 0.0,
  'thic3': 0.0,
  'zpnd': 0.0,
  'rcan': 0.0,
  'scan': 0.0,
  'sno': '_1SNO',
  'albs': 0.0,
  'rhos': '_1RHOS',
  'gro': 1.0},
 2: {'class': 'broadleaf',
  'fcan': 1.0,
  'lamx': 4.148,
  'lnz0': 0.475,
  'lamn': 0.512,
  'alvc': 

In [7]:
c.model.parameter_constraints

{'class': {1: ['clay1', 'sand1', 'clay3'], 3: ['clay1', 'clay2']}}

In [8]:
c.model.parameter_bounds['routing']

{1: {'r2n': [0.0, 1.0], 'r1n': [0.0, 1.0]},
 3: {'pwr': [0.0, 1.0], 'flz': [0.0, 1.0]}}

In [9]:
c.model.parameter_constraints

{'class': {1: ['clay1', 'sand1', 'clay3'], 3: ['clay1', 'clay2']}}

In [10]:
c.calibration.generate_optimizer_templates('/Users/kasrakeshavarz/Downloads/test')

/Users/kasrakeshavarz/Documents/github-repos/fiatmodel/src/fiatmodel/calibration/ostrich/templating.py:200: UserWarning: The directory /Users/kasrakeshavarz/Downloads/test already exists. Contents may be overwritten.
  warnings.warn(f"The directory {path} already exists."


In [11]:
c.calibration.generate_parameter_templates('/Users/kasrakeshavarz/Downloads/test/')

/Users/kasrakeshavarz/Documents/github-repos/fiatmodel/src/fiatmodel/calibration/ostrich/templating.py:200: UserWarning: The directory /Users/kasrakeshavarz/Downloads/test/etc/templates already exists. Contents may be overwritten.
  warnings.warn(f"The directory {path} already exists."


In [12]:
c.calibration.generate_etc_templates('/Users/kasrakeshavarz/Downloads/test/')

/Users/kasrakeshavarz/Documents/github-repos/fiatmodel/src/fiatmodel/calibration/ostrich/templating.py:200: UserWarning: The directory /Users/kasrakeshavarz/Downloads/test/etc already exists. Contents may be overwritten.
  warnings.warn(f"The directory {path} already exists."
/Users/kasrakeshavarz/Documents/github-repos/fiatmodel/src/fiatmodel/calibration/ostrich/templating.py:200: UserWarning: The directory /Users/kasrakeshavarz/Downloads/test/etc/scripts already exists. Contents may be overwritten.
  warnings.warn(f"The directory {path} already exists."
/Users/kasrakeshavarz/Documents/github-repos/fiatmodel/src/fiatmodel/calibration/ostrich/templating.py:200: UserWarning: The directory /Users/kasrakeshavarz/Downloads/test/etc/eval already exists. Contents may be overwritten.
  warnings.warn(f"The directory {path} already exists."
/Users/kasrakeshavarz/Documents/github-repos/fiatmodel/src/fiatmodel/calibration/ostrich/templating.py:200: UserWarning: The directory /Users/kasrakeshavarz